In [1]:
import math
import pandas as pd
import numpy as np
import os

In [2]:
covid_gov_df = pd.read_excel('./dataframes/HIST_PAINEL_COVIDBR_29jul2020.xlsx')

In [3]:
covid_gov_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696368 entries, 0 to 696367
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   regiao                  696368 non-null  object        
 1   estado                  696212 non-null  object        
 2   municipio               689375 non-null  object        
 3   coduf                   696368 non-null  int64         
 4   codmun                  692000 non-null  float64       
 5   codRegiaoSaude          689375 non-null  float64       
 6   nomeRegiaoSaude         689375 non-null  object        
 7   data                    696368 non-null  datetime64[ns]
 8   semanaEpi               696368 non-null  int64         
 9   populacaoTCU2019        693743 non-null  object        
 10  casosAcumulado          696368 non-null  int64         
 11  casosNovos              696368 non-null  int64         
 12  obitosAcumulado         696368

In [4]:
covid_gov_df[covid_gov_df.municipio.notnull()]

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
4493,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-27,13,22945,0,0,0,0,NaN,NaN,0.0
4494,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-28,13,22945,0,0,0,0,NaN,NaN,0.0
4495,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-29,14,22945,0,0,0,0,NaN,NaN,0.0
4496,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-30,14,22945,0,0,0,0,NaN,NaN,0.0
4497,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2020-03-31,14,22945,0,0,0,0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696363,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-25,30,3015268,94187,1773,1275,31,NaN,NaN,1.0
696364,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-26,31,3015268,96332,2145,1308,33,NaN,NaN,1.0
696365,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-27,31,3015268,98480,2148,1339,31,NaN,NaN,1.0
696366,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-28,31,3015268,100726,2246,1391,52,NaN,NaN,1.0


In [5]:
covid_gov_df.municipio.value_counts()

Bom Jesus                625
São Domingos             500
São Francisco            500
Santa Inês               500
Bonito                   500
                        ... 
Santa Tereza de Goiás    125
Monte Belo do Sul        125
Nazarezinho              125
São Miguel do Tapuio     125
Alvorada do Norte        125
Name: municipio, Length: 5246, dtype: int64

In [6]:
covid_gov_df[covid_gov_df.municipio == 'Itaquaquecetuba']

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
446368,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-03-27,13,370821,0,0,0,0,NaN,NaN,1.0
446369,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-03-28,13,370821,0,0,0,0,NaN,NaN,1.0
446370,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-03-29,14,370821,0,0,0,0,NaN,NaN,1.0
446371,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-03-30,14,370821,0,0,0,0,NaN,NaN,1.0
446372,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-03-31,14,370821,1,1,0,0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446488,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-07-25,30,370821,2347,62,187,3,NaN,NaN,1.0
446489,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-07-26,31,370821,2363,16,187,0,NaN,NaN,1.0
446490,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-07-27,31,370821,2375,12,188,1,NaN,NaN,1.0
446491,Sudeste,SP,Itaquaquecetuba,35,352310.0,35011.0,ALTO DO TIETE,2020-07-28,31,370821,2375,0,188,0,NaN,NaN,1.0


See in which location your term was most popular during the specified time frame. Values are calculated on a scale from 0 to 100, where 100 is the location with the most popularity as a fraction of total searches in that location, a value of 50 indicates a location which is half as popular. A value of 0 indicates a location where there was not enough data for this term.

Note: A higher value means a higher proportion of all queries, not a higher absolute query count. So a tiny country where 80% of the queries are for "bananas" will get twice the score of a giant country where only 40% of the queries are for "bananas".

google deleta os dados por hora após uma semana

por motivo de comparação começou com google como benchmark, mas mudou-se para mercado pois este é incrivelmente mais estavel e menor popularidade

In [7]:
path = './dataframes/gtrends_cities_keywords/'
dfs = {}
for folder in os.listdir(path):
    dfs[folder] = pd.DataFrame()
    for file_name in os.listdir(path + folder):
        with open(path+folder+'/'+file_name) as file:
            file.readline()
            df_keyword = pd.read_csv(file,delimiter=',',index_col=0)
            dfs[folder][file_name.split('.')[0]] = df_keyword.iloc[:,1].str.replace(r'\D', '').astype(float)/df_keyword.iloc[:,0].str.replace(r'\D', '').astype(float)

In [8]:
for i in dfs:
    print(i + ':')
    display(dfs[i])

07-14:


,academia,bar,cursos,escola,escritório,feira,hotel,material de construção,restaurante,salão,supermercado
City,,,,,,,,,,,
Embu das Artes,0.086957,0.030928,0.098901,0.219512,0.020408,0.052632,0.041667,0.063830,0.123596,NaN,0.123596
Itaquaquecetuba,0.063830,0.030928,0.149425,0.149425,0.010101,0.052632,0.041667,0.010101,0.063830,0.010101,0.086957
Cotia,0.063830,0.052632,0.136364,0.176471,0.020408,0.075269,0.086957,0.030928,0.176471,0.020408,0.052632
Ribeirão das Neves,0.063830,0.041667,0.063830,0.162791,0.010101,0.052632,0.075269,0.010101,0.063830,0.010101,0.123596
Santana de Parnaíba,0.063830,0.041667,0.136364,0.136364,NaN,0.041667,0.136364,0.020408,0.086957,NaN,0.063830
...,...,...,...,...,...,...,...,...,...,...,...
Goiânia,0.123596,0.111111,0.190476,0.176471,0.041667,0.086957,0.282051,0.041667,0.250000,0.020408,0.315789
Recife,0.149425,0.086957,0.204819,0.333333,0.030928,0.098901,0.351351,0.010101,0.204819,0.020408,0.075269
Belém,0.204819,0.123596,0.219512,0.315789,0.052632,0.098901,0.388889,0.041667,0.219512,0.020408,0.111111


15-21:


,academia,bar,cursos,escola,escritório,feira,hotel,material de construção,restaurante,salão,supermercado
City,,,,,,,,,,,
Rio Grande,0.075269,0.020408,0.098901,0.111111,0.020408,0.020408,0.075269,0.030928,0.020408,0.020408,0.063830
Santa Luzia,0.075269,0.030928,0.111111,0.219512,0.030928,0.086957,0.075269,0.030928,0.086957,NaN,0.123596
Cotia,0.086957,0.063830,0.063830,0.219512,0.020408,0.075269,0.111111,0.020408,0.176471,0.010101,0.075269
Suzano,0.075269,0.030928,0.098901,0.149425,0.041667,0.111111,0.176471,0.010101,0.136364,0.030928,0.052632
Presidente Prudente,0.063830,0.063830,0.086957,0.190476,0.041667,0.086957,0.136364,0.052632,0.149425,0.010101,0.111111
...,...,...,...,...,...,...,...,...,...,...,...
São Luís,0.162791,0.149425,0.282051,0.408451,0.030928,0.075269,0.298701,0.020408,0.162791,0.030928,0.086957
João Pessoa,0.136364,0.098901,0.234568,0.333333,0.052632,0.111111,0.250000,0.052632,0.176471,0.052632,0.176471
Belém,0.219512,0.136364,0.219512,0.333333,0.041667,0.111111,0.515152,0.052632,0.204819,0.052632,0.098901


22-28:


,academia,bar,cursos,escola,escritório,feira,hotel,material de construção,restaurante,salão,supermercado
City,,,,,,,,,,,
Rio Grande,0.063830,0.041667,0.041667,0.063830,0.052632,0.030928,0.063830,NaN,0.030928,0.020408,0.123596
Itaquaquecetuba,0.086957,0.063830,0.111111,0.136364,0.020408,0.098901,0.041667,0.010101,0.075269,0.010101,0.052632
Arujá,0.063830,0.052632,0.190476,0.190476,0.020408,0.111111,0.086957,0.030928,0.086957,0.030928,0.075269
Franca,0.041667,0.063830,0.052632,0.136364,0.020408,0.176471,0.190476,0.020408,0.136364,0.041667,0.190476
Itapevi,0.098901,0.030928,0.123596,0.219512,NaN,0.123596,0.136364,0.030928,0.149425,0.041667,0.162791
...,...,...,...,...,...,...,...,...,...,...,...
João Pessoa,0.190476,0.136364,0.176471,0.388889,0.063830,0.136364,0.282051,0.063830,0.149425,0.030928,0.219512
Natal,0.250000,0.149425,0.265823,0.333333,0.075269,0.098901,0.388889,0.075269,0.250000,0.063830,0.162791
Belém,0.190476,0.123596,0.219512,0.298701,0.052632,0.123596,0.428571,0.052632,0.204819,0.030928,0.149425


In [9]:
df_trends = pd.DataFrame()
for i in dfs:
    aux_df = dfs[i].reset_index()
    aux_df['Date'] = i
    aux_df['_start_day'], aux_df['_end_day'] = i.split('-')
    df_trends = df_trends.append(aux_df, ignore_index=True)

In [10]:
df_trends.set_index(['Date', 'City'], inplace=True)

In [11]:
df_trends

academia       bar    cursos    escola  escritório  \
Date  City                                                                      
07-14 Embu das Artes       0.086957  0.030928  0.098901  0.219512    0.020408   
      Itaquaquecetuba      0.063830  0.030928  0.149425  0.149425    0.010101   
      Cotia                0.063830  0.052632  0.136364  0.176471    0.020408   
      Ribeirão das Neves   0.063830  0.041667  0.063830  0.162791    0.010101   
      Santana de Parnaíba  0.063830  0.041667  0.136364  0.136364         NaN   
...                             ...       ...       ...       ...         ...   
22-28 João Pessoa          0.190476  0.136364  0.176471  0.388889    0.063830   
      Natal                0.250000  0.149425  0.265823  0.333333    0.075269   
      Belém                0.190476  0.123596  0.219512  0.298701    0.052632   
      Fortaleza            0.333333  0.136364  0.204819  0.408451    0.063830   
      Manaus               0.282051  0.098901  0.428571  0.515152    0.063830   

                              feira     hotel  material de construção  \
Date  City                                                              
07-14 Embu das Artes       0.052632  0.041667                0.063830   
      Itaquaquecetuba      0.052632  0.041667                0.010101   
      Cotia                0.075269  0.086957                0.030928   
      Ribeirão das Neves   0.052632  0.075269                0.010101   
      Santana de Parnaíba  0.041667  0.136364                0.020408   
...                             ...       ...                     ...   
22-28 João Pessoa          0.136364  0.282051                0.063830   
      Natal                0.098901  0.388889                0.075269   
      Belém                0.123596  0.428571                0.052632   
      Fortaleza            0.162791  0.388889                0.010101   
      Manaus               0.176471  0.408451                0.149425   

                           restaurante     salão  supermercado _start_day  \
Date  City                                                                  
07-14 Embu das Artes          0.123596       NaN      0.123596         07   
      Itaquaquecetuba         0.063830  0.010101      0.086957         07   
      Cotia                   0.176471  0.020408      0.052632         07   
      Ribeirão das Neves      0.063830  0.010101      0.123596         07   
      Santana de Parnaíba     0.086957       NaN      0.063830         07   
...                                ...       ...           ...        ...   
22-28 João Pessoa             0.149425  0.030928      0.219512         22   
      Natal                   0.250000  0.063830      0.162791         22   
      Belém                   0.204819  0.030928      0.149425         22   
      Fortaleza               0.204819  0.052632      0.234568         22   
      Manaus                  0.219512  0.030928      0.204819         22   

                          _end_day  
Date  City                          
07-14 Embu das Artes            14  
      Itaquaquecetuba           14  
      Cotia                     14  
      Ribeirão das Neves        14  
      Santana de Parnaíba       14  
...                            ...  
22-28 João Pessoa               28  
      Natal                     28  
      Belém                     28  
      Fortaleza                 28  
      Manaus                    28  

[271 rows x 13 columns]

In [12]:
df_trends.to_csv('./dataframes/df_trends.csv')

In [13]:
df_trends.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 271 entries, ('07-14', 'Embu das Artes') to ('22-28', 'Manaus')
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   academia                271 non-null    float64
 1   bar                     271 non-null    float64
 2   cursos                  271 non-null    float64
 3   escola                  271 non-null    float64
 4   escritório              268 non-null    float64
 5   feira                   271 non-null    float64
 6   hotel                   271 non-null    float64
 7   material de construção  262 non-null    float64
 8   restaurante             271 non-null    float64
 9   salão                   259 non-null    float64
 10  supermercado            271 non-null    float64
 11  _start_day              271 non-null    object 
 12  _end_day                271 non-null    object 
dtypes: float64(11), object(2)
memory usage: 29.0+ KB


In [14]:
covid_gov_df.loc[(covid_gov_df.municipio.isin(df_trends.index.get_level_values(1).unique().values.tolist())) & 
                 (covid_gov_df.data >= '07-07-2020') & 
                 (covid_gov_df.data <= '14-07-2020')].groupby('municipio').casosNovos.sum()

municipio
Americana                420
Aparecida de Goiânia    1002
Aracaju                 3811
Araraquara               205
Arujá                    105
                        ... 
Uberaba                  207
Uberlândia              1671
Vila Velha              1426
Vitória da Conquista     355
Volta Redonda            217
Name: casosNovos, Length: 106, dtype: int64

In [15]:
aux = []
for i in df_trends.index.get_level_values(0).unique().values:
    aux.append(covid_gov_df.loc[(covid_gov_df.municipio.isin(df_trends.index.get_level_values(1).unique().values.tolist())) & 
                                 (covid_gov_df.data >= i.split('-')[0] + '-07-2020') & 
                                 (covid_gov_df.data <= i.split('-')[1] + '-07-2020')].groupby('municipio'))

In [17]:
column = 'casosNovos'
method = sum

aux_df = pd.DataFrame(pd.concat(map(lambda x : x['casosNovos'].apply(method), aux), keys=df_trends.index.get_level_values(0).unique().values))
aux_df.index = aux_df.index.rename(['Date', 'City'])
df_trends.join(aux_df)

academia       bar    cursos    escola  escritório  \
Date  City                                                                      
07-14 Embu das Artes       0.086957  0.030928  0.098901  0.219512    0.020408   
      Itaquaquecetuba      0.063830  0.030928  0.149425  0.149425    0.010101   
      Cotia                0.063830  0.052632  0.136364  0.176471    0.020408   
      Ribeirão das Neves   0.063830  0.041667  0.063830  0.162791    0.010101   
      Santana de Parnaíba  0.063830  0.041667  0.136364  0.136364         NaN   
...                             ...       ...       ...       ...         ...   
22-28 João Pessoa          0.190476  0.136364  0.176471  0.388889    0.063830   
      Natal                0.250000  0.149425  0.265823  0.333333    0.075269   
      Belém                0.190476  0.123596  0.219512  0.298701    0.052632   
      Fortaleza            0.333333  0.136364  0.204819  0.408451    0.063830   
      Manaus               0.282051  0.098901  0.428571  0.515152    0.063830   

                              feira     hotel  material de construção  \
Date  City                                                              
07-14 Embu das Artes       0.052632  0.041667                0.063830   
      Itaquaquecetuba      0.052632  0.041667                0.010101   
      Cotia                0.075269  0.086957                0.030928   
      Ribeirão das Neves   0.052632  0.075269                0.010101   
      Santana de Parnaíba  0.041667  0.136364                0.020408   
...                             ...       ...                     ...   
22-28 João Pessoa          0.136364  0.282051                0.063830   
      Natal                0.098901  0.388889                0.075269   
      Belém                0.123596  0.428571                0.052632   
      Fortaleza            0.162791  0.388889                0.010101   
      Manaus               0.176471  0.408451                0.149425   

                           restaurante     salão  supermercado _start_day  \
Date  City                                                                  
07-14 Embu das Artes          0.123596       NaN      0.123596         07   
      Itaquaquecetuba         0.063830  0.010101      0.086957         07   
      Cotia                   0.176471  0.020408      0.052632         07   
      Ribeirão das Neves      0.063830  0.010101      0.123596         07   
      Santana de Parnaíba     0.086957       NaN      0.063830         07   
...                                ...       ...           ...        ...   
22-28 João Pessoa             0.149425  0.030928      0.219512         22   
      Natal                   0.250000  0.063830      0.162791         22   
      Belém                   0.204819  0.030928      0.149425         22   
      Fortaleza               0.204819  0.052632      0.234568         22   
      Manaus                  0.219512  0.030928      0.204819         22   

                          _end_day  casosNovos  
Date  City                                      
07-14 Embu das Artes            14       267.0  
      Itaquaquecetuba           14       355.0  
      Cotia                     14       246.0  
      Ribeirão das Neves        14       410.0  
      Santana de Parnaíba       14       153.0  
...                            ...         ...  
22-28 João Pessoa               28      2416.0  
      Natal                     28      1374.0  
      Belém                     28       922.0  
      Fortaleza                 28      1501.0  
      Manaus                    28      2008.0  

[271 rows x 14 columns]

---